In [ ]:
import pandas as pd
import logging
import os
import pyodbc
import shutil
from sqlalchemy import create_engine, text
from sqlalchemy.types import UnicodeText

# --NƠI VIẾT CÁC HÀM--

# Xác định loại file Excel (MN: miền nam, MB: miền bắc, MT: miền trung)
def find_matching_key(json_data, text):
    text_lower = text.lower().strip()

    for key, values in json_data.items():
        for value in values:
            value_lower = value.lower().strip()

            if key == "MN":
                if value_lower in text_lower:
                    return key
            else:  # MB, MT
                if text_lower.startswith(value_lower):
                    return key

    return None

# Chuyển text thành số thực
def convert_to_float(value):
    try:
        # Convert ',' to '.' if present
        if isinstance(value, str):
            value = value.replace(',', '.')

        # Convert to float, return 0 if not possible
        return float(value) if pd.notnull(value) else 0.0
    except (ValueError, TypeError):
        return 0.0

# Chuyên xử lý MB
def mb_sheet(text, search_list):
    """
    Tìm giá trị đầu tiên trong search_list có chứa trong text

    Args:
        text (str): Chuỗi cần kiểm tra
        search_list (list): Danh sách các giá trị cần tìm

    Returns:
        str or None: Giá trị đầu tiên tìm thấy hoặc None nếu không tìm thấy
    """
    if pd.isna(text):  # Xử lý trường hợp NaN
        return None

    text_str = str(text).upper()

    for value in search_list:
        if str(value).upper() in text_str:
            return value

    return None

# Đọc + xử lý data từ file Excel
def process_excel_files(file_name , data_folder , json_data , search_list):
    combined_data = []
    columns_to_extract = [
        'flightdate', 'flightno', 'actype', 'route',
        'cgo', 'mail',  'seat', 'adl', 'chd', 'totalpax',
        'acregno', 'source', 'sheet_name'
    ]
    file_path =  data_folder

    # Convert col sang chữ thường
    columns_to_extract = [col.lower() for col in columns_to_extract]
    numeric_columns = ['cgo', 'mail', 'adl', 'chd' , 'seat']

    # Iterate through files in the directory
        # Filter files containing 'update' and 'toan cang'
    if find_matching_key ( json_data , file_name) == "MN":
        full_file_path = os.path.join(file_path, file_name)

        try:
            # Read all sheets in the Excel file
            excel_sheets = pd.read_excel(full_file_path, sheet_name=None)

            # Loop each sheet
            for sheet_name, df_sheet in excel_sheets.items():
                # Process only sheets with 3-character names
                if len(sheet_name) >= 1:
                    # Lowercase column names
                    df_sheet.columns = df_sheet.columns.str.lower()

                    # Add missing columns with None/NaN
                    for col in columns_to_extract:
                        if col not in df_sheet.columns:
                            df_sheet[col] = pd.NA

                    # Add metadata columns
                    df_sheet['source'] = file_name
                    df_sheet['sheet_name'] = sheet_name

                    # Handle specific column data types and fill missing values
                    # ffill(): điền value thiếu (NaN/None) = value gần nhất phía trên nó
                    # fillna(''): điền value thiếu (NaN/None) = chuỗi rỗng + astype(): ép kiểu
                    df_sheet['flightdate'] = df_sheet['flightdate'].ffill()
                    df_sheet['flightno'] = df_sheet['flightno'].fillna('').astype(str)
                    df_sheet['actype'] = df_sheet['actype'].fillna('').astype(str)
                    df_sheet['route'] = df_sheet['route'].fillna('').astype(str)

                    # Convert numeric columns to float
                    for col in numeric_columns:
                        try:
                            df_sheet[col] = df_sheet[col].apply(convert_to_float)

                            # Fallback if couldn't convert NaN/None
                            df_sheet[col] = df_sheet[col].fillna(0)
                        except Exception as e:
                            logging.error(f"Error processing numeric column '{col}' in sheet '{sheet_name}' from file '{file_name}': {e}. Attempting to fill with 0.")
                            df_sheet[col] = df_sheet[col].fillna(0) # Fallback

                    # Extract specified columns
                    extracted_df = df_sheet[columns_to_extract].copy()
                    combined_data.append(extracted_df)

        except Exception as e:
            logging.error(f"Error processing file {file_name}: {e}")

    elif  find_matching_key ( json_data , file_name) == "MT" :
            full_file_path = os.path.join(file_path, file_name)

            try:
                # Read all sheets in the Excel file
                excel_sheets = pd.read_excel(full_file_path, sheet_name=None)

                numeric_columns = ['cgo', 'mail', 'adl', 'chd' , 'totalpax','seat']
                for sheet_name, df_sheet in excel_sheets.items():
                    # Process only sheets with 3-character names
                    if len(sheet_name) >= 1:
                        # Lowercase column names
                        df_sheet.columns = df_sheet.columns.str.lower()

                        # Add missing columns with None/NaN
                        for col in columns_to_extract:
                            if col not in df_sheet.columns:
                                df_sheet[col] = pd.NA

                        # Add metadata columns
                        df_sheet['source'] = file_name
                        df_sheet['sheet_name'] = sheet_name

                        # Handle specific column data types and fill missing values
                        df_sheet['flightdate'] = df_sheet['flightdate'].ffill()
                        df_sheet['flightno'] = df_sheet['flightno'].fillna('').astype(str)
                        df_sheet['actype'] = df_sheet['actype'].fillna('').astype(str)
                        df_sheet['route'] = df_sheet['route'].fillna('').astype(str)

                        # Convert numeric columns using custom conversion function
                        for col in numeric_columns:
                            df_sheet[col] = df_sheet[col].apply(convert_to_float)

                        # Extract specified columns
                        extracted_df = df_sheet[columns_to_extract].copy()
                        combined_data.append(extracted_df)

            except Exception as e:
                logging.error(f"Error processing file {file_name}: {e}")

    elif  find_matching_key ( json_data , file_name) == "MB" :
            full_file_path = os.path.join(file_path, file_name)

            try:
                # Read all sheets in the Excel file
                print ( "MB: file " + file_name )
                excel_sheets = pd.read_excel(full_file_path, sheet_name=None)

                numeric_columns = ['cgo', 'mail', 'adl', 'chd' , 'seat' , 'totalpax']
                for sheet_name, df_sheet in excel_sheets.items():
                    # Process only sheets with 3-character names
                    if len(sheet_name) >= 1:
                        # Lowercase column names
                        df_sheet.columns = df_sheet.columns.str.lower()

                        # Add missing columns with None/NaN
                        for col in columns_to_extract:
                            if col not in df_sheet.columns:
                                df_sheet[col] = pd.NA

                        # Add metadata columns
                        df_sheet['source'] = file_name
                        # Handle specific column data types and fill missing values
                        df_sheet['flightdate'] = df_sheet['flightdate'].ffill()
                        df_sheet['flightno'] = df_sheet['flightno'].fillna('').astype(str)
                        df_sheet['actype'] = df_sheet['actype'].fillna('').astype(str)
                        df_sheet['route'] = df_sheet['route'].fillna('').astype(str)
                        df_sheet['sheet_name'] = df_sheet['route'].apply( lambda x: mb_sheet(x, search_list))

                        # Convert numeric columns using custom conversion function
                        for col in numeric_columns:
                            df_sheet[col] = df_sheet[col].apply(convert_to_float)

                        # Extract specified columns
                        extracted_df = df_sheet[columns_to_extract].copy()
                        combined_data.append(extracted_df)

            except Exception as e:
                logging.error(f"Error processing file {file_name}: {e}")

    # Combine all DataFrames
    else:
        print( "Error")

    if combined_data:
        final_df = pd.concat(combined_data, ignore_index=True)
        return final_df
    else:
        logging.warning("No data extracted from files.")
        return pd.DataFrame()

def connect_to_sql_server(database , server):
    try:
        connection_string = (
            f"mssql+pyodbc://@{server}/{database}"
            "?driver=ODBC+Driver+17+for+SQL+Server"
        )
        engine = create_engine(connection_string)
        print("Connected to SQL Server successfully!")
        return engine
    except Exception as e:
        print(f"Error connecting to SQL Server: {str(e)}")
        return None

# Hàm kiểm tra file đã import chưa
def is_file_imported(con , file_name):
    if con :
        # with engine.connect() as con:
        query = text("SELECT 1 FROM import_log WHERE file_name = :file_name")
        result = con.execute(query, {"file_name": file_name}).fetchone()
        return result is not None
    return False

# Hàm đánh dấu file đã import
def mark_file_imported(con, file_name, source_type, row_count):
    print('log row')
    insert_query = text("""
        INSERT INTO import_log (file_name, source_type, row_count)
        VALUES (:file_name, :source_type, :row_count)
    """)
    con.execute(insert_query, {
        "file_name": file_name,
        "source_type": source_type,
        "row_count": row_count
    })

# Func điều phối việc import các file Excel
def import_file (engine , data_folder , destination_folder , json_data , search_list, ):
    if engine:
        with engine.begin() as con:
            for file in os.listdir(data_folder) :
                if not is_file_imported(con, file):
                    print(f"Đang xử lý: {file}")
                    try:
                        if  str(file).find('xl') > 0:
                            df = process_excel_files(file , data_folder , json_data , search_list)
                            print( len (df ))
                            row_count = len(df)

                            # Ghi data vào table 'flight_raw'
                            df[df["flightno"].notna() & df["actype"].notna()][
                                [
                                    "flightdate",
                                    "flightno",
                                    "route",
                                    "actype",
                                    "seat",
                                    "adl",
                                    "chd",
                                    "cgo",
                                    "mail",
                                    "totalpax",
                                    "source",
                                    "acregno",
                                    "sheet_name",
                                ]
                            ].to_sql(
                                "flight_raw",
                                con=conn,
                                if_exists="append",
                                index=False,
                                dtype={
                                    "source": UnicodeText(),
                                    "sheet_name": UnicodeText(),
                                    "flightdate": UnicodeText(),
                                },
                            )

                            # Ghi lại thông tin import vào bảng import_log
                            mark_file_imported(con, file, 'type1', row_count)

                            # Di chuyển tệp đã xử lý từ data_folder sang destination_folder
                            file_path = os.path.join(data_folder, file)
                            shutil.move(file_path, os.path.join(destination_folder, file))

                            print(f"Đã import file: {file}")
                    except Exception as e:
                        print(f"Lỗi khi xử lý file {file}: {e}")
                else:
                    print(f"{file} đã được import trước đó, bỏ qua.")

: 

## Chay dong dau truoc

In [ ]:
# --NƠI THỰC THI--

json_data = {
    "MN": ["toan cang"], # chua ten bien
    "MB": ["NAA"], # bat dau
    "MT": ["CV1"] # bat dau
}

# Ds các actype
search_list =  [
  "THD",
  "HPH",
  "HAN",
  "VDO",
  "VDH",
  "VII",
  "DIN"
]

# Kết nối SQL
server = 'DESKTOP-MC6MTTM\\SQLEXPRESS'
engine = connect_to_sql_server( database= 'flight' , server= server)


# Đường dẫn đến folder chứa các file Excel cần xử lý
data_folder = r"D:\Code Python\Format_data"

# Đường dẫn đến folder để lưu trữ các file đã xử lý
destination_folder = r"D:\Code Python\imported_file"

import_file( engine= engine , data_folder= data_folder , destination_folder= destination_folder ,json_data= json_data, search_list = search_list)


Connected to SQL Server successfully!
Đang xử lý: CV1 Số liệu toan cang tuan 28.2025.xlsx
6336
log row
Đã import file: CV1 Số liệu toan cang tuan 28.2025.xlsx
Đang xử lý: CV1 Số liệu toan cang tuan 29.2025.xlsx
6409
log row
Đã import file: CV1 Số liệu toan cang tuan 29.2025.xlsx
Đang xử lý: CV1 Số liệu toan cang tuan 30.2025.xlsx
6422
log row
Đã import file: CV1 Số liệu toan cang tuan 30.2025.xlsx
Đang xử lý: CV1 Số liệu toan cang tuan 31.2025.xlsx
6284
log row
Đã import file: CV1 Số liệu toan cang tuan 31.2025.xlsx
Đang xử lý: CV1 TUAN 28 2025.xlsx
3898
log row
Đã import file: CV1 TUAN 28 2025.xlsx
Đang xử lý: CV1 TUAN 29 2025.xlsx
3879
log row
Đã import file: CV1 TUAN 29 2025.xlsx
Đang xử lý: CV1 TUAN 30 2025.xlsx
3861
log row
Đã import file: CV1 TUAN 30 2025.xlsx
Đang xử lý: CV1 TUAN 31 2025.xlsx
3720
log row
Đã import file: CV1 TUAN 31 2025.xlsx
Đang xử lý: Final_import_2.ipynb
Đang xử lý: NAA_CV1 Tuần 28.2025.xlsx
MB: file NAA_CV1 Tuần 28.2025.xlsx
5218
log row
Đã import file: NAA

## Trả kết quả ra file thiếu route và actype chứa seat

In [ ]:
import pyodbc
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import UnicodeText
from openpyxl import load_workbook
# Kết nối tới SQL Server qua SQLAlchemy


server = 'DESKTOP-MC6MTTM\\SQLEXPRESS'
def connect_to_sql_server(database , server):
    try:
        connection_string = (
            f"mssql+pyodbc://@{server}/{database}"
            "?driver=ODBC+Driver+17+for+SQL+Server"
        )
        engine = create_engine(connection_string)
        print("Connected to SQL Server successfully!")
        return engine
    except Exception as e:
        print(f"Error connecting to SQL Server: {str(e)}")
        return None

con = connect_to_sql_server( server= server, database='flight')
query1 = '''
select top 0 *
from actype_seat
union all
select Value , null
from Missing_Dimensions_Log
where type = 'Actype'
group by Value
'''

query2 = '''
select top 0
    [ROUTE],
    [AC],
    [Route_ID],
    [FLIGHT HOUR],
    [TAXI],
    [BLOCK HOUR],
    [DISTANCE KM],
    [Loại],
    [Type],
    [Country]
from Route
union
select  Value ,NULL AS from_airport,
    NULL AS to_airport,
    NULL AS route_type,
    NULL AS cgo,
    NULL AS pax,
    NULL AS mail,
    NULL AS airline_code,
    NULL AS note,
    NULL AS created_by
from Missing_Dimensions_Log
where type = 'Route'
and value is not null
'''

query3 = '''
select top 0
    [ROUTE],
    [Distance mile GDS],
    [Distance km GDS],
    [Sector_2],
    [Country 1],
    [Country 2],
    [Country],
    [DOM/INT],
    [Area]
from Airline_Route_Details
union
select  Value ,
	NULL  ,
    NULL ,
    NULL ,
    NULL,
    NULL ,
    NULL,
    NULL ,
    NULL
from Missing_Dimensions_Log
where type = 'Route'
and value is not null
'''

# Đọc kết quả
df1 = pd.read_sql(query1, con )
df2 = pd.read_sql(query2, con )
df3 = pd.read_sql(query3, con )


with pd.ExcelWriter( r"D:\Code Python\Add_information.xlsx", engine='openpyxl' ) as writer:
    df1.to_excel(writer, sheet_name='Actype_seat', index=False)
    df2.to_excel(writer, sheet_name='Route', index=False)
    df3.to_excel(writer, sheet_name='Airline_Route_Details', index=False)

Connected to SQL Server successfully!


## Thêm các giá trị còn thiếu

In [ ]:
from sqlalchemy import create_engine, text
engine = connect_to_sql_server( con , 'flight')

actype = pd.read_excel ( 'Add_information.xlsx', sheet_name="Actype_seat")
route= pd.read_excel ( 'Add_information.xlsx', sheet_name="Route")
# route_detail= pd.read_excel ( 'Add_information.xlsx', sheet_name="Route")
route_detail= pd.read_excel ( 'Add_information.xlsx', sheet_name="Airline_Route_Details")

# Drop các row chứa bất kỳ value thiếu (NaN/None)
actype.dropna()
route.dropna()
route_detail.dropna()

actype[actype['seat'].notnull()].to_sql( 'TempActypeImport' , if_exists="append", con= engine , index = False)
route[route['Country'].notnull()].to_sql( 'TempRouteImport' , if_exists="append", con = engine , index = False)

# Stored procedure sẽ lấy data từ các bảng tạm TempActypeImport và TempRouteImport để cập nhật/thêm mới data vào các bảng chính
def execute_stored_procedure(engine, procedure_name):
    """
    Execute stored procedure using SQLAlchemy 2.0 engine
    """
    try:
        with engine.begin() as connection:  # auto-commit context
            print(f"Executing stored procedure {procedure_name}...")
            connection.execute(text(f"EXEC {procedure_name}"))
            print("Stored procedure executed successfully!")
    except Exception as e:
        print(f"Error executing stored procedure: {e}")

# Kết nối và thực thi
if engine:
    execute_stored_procedure(engine, 'dbo.usp_ImportAndUpdateMissingDimensions')



Connected to SQL Server successfully!


InterfaceError: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)

In [ ]:

actype = pd.read_excel ( 'output.xlsx', sheet_name="Actype")
route= pd.read_excel ( 'output.xlsx', sheet_name="Route")
actype.dropna()
route.dropna()
# actype[actype['seat'].notnull()]
# .to_sql( 'TempActypeImport' , if_exists="append", con= engine , index = False)
route[route['Country'].notnull()]

,ROUTE,AC,Route_ID,FH (THEO GIỜ),FLIGHT HOUR,TAXI,BLOCK HOUR,DISTANCE KM,Loại,Type,Country
0,AKL-HAN,Wide_body,AKLHANWide_body,11,11,0.5,11.5,9200,INT,Long_haul,NZ-VN


In [ ]:
type = find_matching_key (json_data , "CV1 TUAN 51 2024.xlsx")
type2 = find_matching_key (json_data , "NAA CV1 tuan 3 2025.xlsx")
print( type )
print( type2 )
# actype = pd.read_excel ( 'output.xlsx', sheet_name="Actype")
# route= pd.read_excel ( 'output.xlsx', sheet_name="Route")
# route

MB
MT
